In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp, col, count, countDistinct, sum, avg
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DoubleType
from delta.tables import *
from delta import *
from delta import configure_spark_with_delta_pip


In [2]:
builder = SparkSession \
        .builder \
        .appName('healthcare_Aggregations')\
        .master('local')\
        .config("spark.driver.memory","2g")\
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")\
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
        .config("spark.sql.execution.arrow.pyspark.enabled", "true")


spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [4]:
source = spark.read \
                .format("delta").load("silver\\healthcare_trans")             

In [ ]:
source.printSchema()

In [7]:
source.createOrReplaceTempView("health")

In [ ]:
#Insurance Analysis
pop_insurance = spark.sql("SELECT PAYER_NAME, \
                          SUM(TOTAL_CLAIM_COST) AS TOTALCLAIMCOST,\
                          AVG(PAYER_COVERAGE) AS AVERAGEPAYERCOVERAGE,\
                          COUNT(ENCOUNTERCLASS) AS TOTAL_CLAIMS\
                          FROM health \
                          GROUP BY PAYER_NAME \
                          ORDER BY AVERAGEPAYERCOVERAGE DESC")
pop_insurance.show()

In [ ]:
#Common Encounters
encount_cases = spark.sql("SELECT ENCOUNTERCLASS, \
                          COUNT(ENCOUNTERCLASS) AS OCCURENCES,\
                          SUM(BASE_ENCOUNTER_COST) AS TOTAL_BASE_ENCOUNTER_COST,\
                          SUM(BASE_COST) AS TOTAL_BASE_COST\
                          FROM health \
                          GROUP BY ENCOUNTERCLASS \
                          ORDER BY OCCURENCES DESC")
encount_cases.show()

In [ ]:
spark.stop